# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

#Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../output_data/cities.csv")
city_weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ponta do sol,78,PT,1632798348,83,38.7333,-28.2667,64.42,14.03
1,1,namibe,30,AO,1632798348,89,-15.1961,12.1522,62.13,3.87
2,2,biak,95,ID,1632798349,68,-0.9131,122.8766,83.07,4.07
3,3,ushuaia,93,AR,1632798349,94,-54.8000,-68.3000,36.90,3.15
4,4,rikitea,37,PF,1632798350,75,-23.1203,-134.9692,73.15,21.14
...,...,...,...,...,...,...,...,...,...,...
549,549,ekibastuz,100,KZ,1632798507,81,51.6667,75.3667,38.43,13.67
550,550,tilichiki,100,RU,1632798507,55,60.4667,166.1000,42.78,8.66
551,551,talara,10,PE,1632798508,87,-4.5772,-81.2719,64.81,12.62
552,552,atemar,100,RU,1632798508,87,54.1810,45.4091,38.05,13.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"].astype(int)

In [4]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=99, point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
fig(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = city_weather_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,16,16,yumen,0,CN,1632798353,13,40.2833,97.2000,71.76,1.52
1,78,78,jiuquan,0,CN,1632798370,14,39.7432,98.5174,71.60,6.76
2,138,138,ginda,0,IN,1632798386,68,30.6945,78.4932,70.57,1.28
3,149,149,qena,0,EG,1632798389,44,26.1615,32.7183,72.90,8.05
4,158,158,pacific grove,0,US,1632798391,74,38.4820,-90.7415,78.98,1.01
5,175,175,sabha,0,LY,1632798397,36,27.0377,14.4283,70.99,7.87
6,208,208,zhangye,0,CN,1632798406,15,38.9342,100.4517,71.85,3.24
7,400,400,praia da vitoria,0,CV,1632798462,78,14.9215,-23.5087,77.54,6.91
8,408,408,mouzouras,0,GR,1632798464,72,35.5333,24.1500,74.43,0.00
9,452,452,tabas,0,IR,1632798477,24,33.5959,56.9244,76.50,1.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='Unnamed: 0')
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,16,yumen,0,CN,1632798353,13,40.2833,97.2000,71.76,1.52,NaN
1,78,jiuquan,0,CN,1632798370,14,39.7432,98.5174,71.60,6.76,NaN
2,138,ginda,0,IN,1632798386,68,30.6945,78.4932,70.57,1.28,NaN
3,149,qena,0,EG,1632798389,44,26.1615,32.7183,72.90,8.05,NaN
4,158,pacific grove,0,US,1632798391,74,38.4820,-90.7415,78.98,1.01,NaN
5,175,sabha,0,LY,1632798397,36,27.0377,14.4283,70.99,7.87,NaN
6,208,zhangye,0,CN,1632798406,15,38.9342,100.4517,71.85,3.24,NaN
7,400,praia da vitoria,0,CV,1632798462,78,14.9215,-23.5087,77.54,6.91,NaN
8,408,mouzouras,0,GR,1632798464,72,35.5333,24.1500,74.43,0.00,NaN
9,452,tabas,0,IR,1632798477,24,33.5959,56.9244,76.50,1.68,NaN


In [7]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,16,yumen,0,CN,1632798353,13,40.2833,97.2000,71.76,1.52,NaN
1,78,jiuquan,0,CN,1632798370,14,39.7432,98.5174,71.60,6.76,Jiuquan Hotel
2,138,ginda,0,IN,1632798386,68,30.6945,78.4932,70.57,1.28,Bal uma sankar hotel and restaurant
3,149,qena,0,EG,1632798389,44,26.1615,32.7183,72.90,8.05,"Grand hotel ""Quena"""
4,158,pacific grove,0,US,1632798391,74,38.4820,-90.7415,78.98,1.01,Comfort Inn Near Six Flags St. Louis
5,175,sabha,0,LY,1632798397,36,27.0377,14.4283,70.99,7.87,Foundation Taaj Asala for Historical Research ...
6,208,zhangye,0,CN,1632798406,15,38.9342,100.4517,71.85,3.24,Zhangye Lvzhou Holiday Inn
7,400,praia da vitoria,0,CV,1632798462,78,14.9215,-23.5087,77.54,6.91,LT Aparthotel
8,408,mouzouras,0,GR,1632798464,72,35.5333,24.1500,74.43,0.00,Areti Suites
9,452,tabas,0,IR,1632798477,24,33.5959,56.9244,76.50,1.68,شرکت معماری مهراز شایگان آراد


In [8]:
#clean up DataSet by remooving NAN
hotel_df = hotel_df.dropna()
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,78,jiuquan,0,CN,1632798370,14,39.7432,98.5174,71.60,6.76,Jiuquan Hotel
2,138,ginda,0,IN,1632798386,68,30.6945,78.4932,70.57,1.28,Bal uma sankar hotel and restaurant
3,149,qena,0,EG,1632798389,44,26.1615,32.7183,72.90,8.05,"Grand hotel ""Quena"""
4,158,pacific grove,0,US,1632798391,74,38.4820,-90.7415,78.98,1.01,Comfort Inn Near Six Flags St. Louis
5,175,sabha,0,LY,1632798397,36,27.0377,14.4283,70.99,7.87,Foundation Taaj Asala for Historical Research ...
6,208,zhangye,0,CN,1632798406,15,38.9342,100.4517,71.85,3.24,Zhangye Lvzhou Holiday Inn
7,400,praia da vitoria,0,CV,1632798462,78,14.9215,-23.5087,77.54,6.91,LT Aparthotel
8,408,mouzouras,0,GR,1632798464,72,35.5333,24.1500,74.43,0.00,Areti Suites
9,452,tabas,0,IR,1632798477,24,33.5959,56.9244,76.50,1.68,شرکت معماری مهراز شایگان آراد
10,460,damietta,0,EG,1632798479,73,31.4165,31.8133,73.74,9.10,SULIMAN INN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
### Credit for code to 'ASooklall' user on Github.com and 'Amos Johnson' user on GitLab ###